In [2]:
using JuMP, Gurobi
  
include("read.jl")
n,d,f,Amin,Nr,R,regions,coords,D = readInstance("instances aerodromes-20221003/aerodrome_30_1.txt")


(30, 19, 10, 14, 3, 18, Dict{Any, Any}(2 => Any[8, 10, 27], 3 => Any[1, 6, 7, 22, 24, 26, 30], 1 => Any[2, 4, 9, 12, 13, 14, 15, 16, 18, 23, 28, 29]), [10 53; 57 8; … ; 25 58; 42 4], [0 65 … 15 58; 65 0 … 59 15; … ; 15 59 … 0 56; 58 15 … 56 0])

In [39]:
#formulation polynomiale

model = Model(Gurobi.Optimizer)
#déclaration des variables
@variable(model, x[1:n,1:n], Bin) #variables binaires -> présence ou non de l'arc i,j
@variable(model,u[1:n]) #variable entières : numéro du sommet i dans le chemin
#déclaration des contraintes
A = Matrix{Int64}(ones(n,n))
b = [1 for i in 1:n]
c = [0 for i in 1:n]
c[d]=1
c[f]=-1
#reecriture
for i in 1:n
    @constraint(model,sum(x[i,j] for j in 1:n)<=1)
    @constraint(model,sum(x[j,i] for j in 1:n)<=1)
end
for i in 1:n 
    if i == d 
        @constraint(model,sum(x[i,j] for j in 1:n)-sum(x[j,i] for j in 1:n) == 1)
    elseif i==f
        @constraint(model,sum(x[i,j] for j in 1:n)-sum(x[j,i] for j in 1:n) == -1)
    else 
        @constraint(model,sum(x[i,j] for j in 1:n)-sum(x[j,i] for j in 1:n) == 0)
    end
end

@constraint(model,sum(sum(x[i,j] for j in 1:n) for i in 1:n)>= Amin-1)
for i in 1:n 
    for j in 1:n 
        if D[i,j]>R 
            @constraint(model,x[i,j]==0)
        end
    end
end

for i in eachindex(regions) 
    @constraint(model, sum(sum(x[k,j] for j in 1:n) for k in regions[i] ) >= 1)
end
for i in 1:n
    for j in 1:n
        if i!=d && i!=f && j!=d && j!=f
            @constraint(model, u[j]>=u[i]+1-Amin*(1-x[i,j]))
        end
    end
end

#objectif
@objective(model, Min, sum(sum(D[i,j]*x[i,j] for i in 1:n) for j in 1:n))
JuMP.optimize!(model)
objective_value = JuMP.objective_value(model)
println("Objective value : ",objective_value)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-09
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1584 rows, 930 columns and 8102 nonzeros
Model fingerprint: 0x9116137d
Variable types: 30 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 784 rows and 748 columns
Presolve time: 0.01s
Presolved: 800 rows, 182 columns, 2648 nonzeros
Variable types: 27 continuous, 155 integer (155 binary)

Root relaxation: objective 5.314286e+01, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   53.14286    0   20          -   53.14286      -

In [5]:

using Plots
function visualisation(x)
    fig = plot()
    labels = ["région $i" for i in 1:Nr]
    scatter!([coords[d,1]],[coords[d,2]], markersize = 10, color =:red, label = "départ")
    scatter!([coords[f,1]],[coords[f,2]], markersize = 10, color =:green, label = "arrivée")
    scatter!(coords[:,1],coords[:,2], label = false)
    for region in 1:Nr
        x_aerodromes_region = []
        y_aerodromes_region = []
        for aerodrome in 1:length(regions[region])
            push!(x_aerodromes_region,coords[regions[region][aerodrome],1])
            push!(y_aerodromes_region,coords[regions[region][aerodrome],2])
        end
        scatter!(x_aerodromes_region,y_aerodromes_region, label = labels[region])
    end
    for i in 1:n
        for j in 1:n
            if x[i,j] == 1
                print(i)
                print(j)
                plot!([coords[i,1],coords[j,1]],[coords[i,2],coords[j,2]], color=:blue, label = false)
            end
        end
    end
    return fig
end

visualisation (generic function with 1 method)

In [ ]:
#formulation exponentielle

contraintes_sous_tours = []
contraintes_violees = true
objective = 1000
x_val = []
X = []

function sous_probleme(x)
    sous_model = Model(Gurobi.Optimizer)
    #déclaration des variables
    @variable(sous_model, a[1:n], Bin)
    @variable(sous_model, b[1:n,1:n])
    @objective(sous_model, Max, sum(x[i,j]*b[i,j] for i in 1:n, j in 1:n) - (sum(a[i] for i in 1:n)- 1))
    @constraint(sous_model, sum(a[i] for i in 1:n) >= 2)
    for i in 1:n
        for j in 1:n
            @constraint(sous_model, b[i,j] >= 0)
            @constraint(sous_model, b[i,j] <= a[i])
            @constraint(sous_model, b[i,j] <= a[j])
            @constraint(sous_model, b[i,j] >= a[i] + a[j] - 1)
        end
    end
    JuMP.optimize!(sous_model) 
    contraintes_violees = (JuMP.objective_value(sous_model) > 0)
    nouvelle_contrainte = JuMP.value.(sous_model[:a])
    return contraintes_violees, nouvelle_contrainte
end

while contraintes_violees
    model = Model(Gurobi.Optimizer)
    #déclaration des variables
    @variable(model, x[1:n,1:n], Bin) #variables binaires -> présence ou non de l'arc i,j
    #déclaration des contraintes
    A = Matrix{Int64}(ones(n,n))
    b = [1 for i in 1:n]
    c = [0 for i in 1:n]
    c[d]=1
    c[f]=-1
    #reecriture
    for i in 1:n
        @constraint(model,sum(x[i,j] for j in 1:n)<=1)
        @constraint(model,sum(x[j,i] for j in 1:n)<=1)
    end

    for i in 1:n
        @constraint(model, x[i,i] == 0)
    end

    #on interdit déjà les sous-tours de taille 2
    for i in 1:n
        for j in 1:n
            @constraint(model, x[i,j] + x[j,i] <= 1)
        end
    end
    
    for i in 1:n 
        if i == d 
            @constraint(model,sum(x[i,j] for j in 1:n)-sum(x[j,i] for j in 1:n) == 1)
        elseif i==f
            @constraint(model,sum(x[i,j] for j in 1:n)-sum(x[j,i] for j in 1:n) == -1)
        else 
            @constraint(model,sum(x[i,j] for j in 1:n)-sum(x[j,i] for j in 1:n) == 0)
        end
    end

    @constraint(model,sum(sum(x[i,j] for j in 1:n) for i in 1:n)>= Amin-1)
    for i in 1:n 
        for j in 1:n 
            if D[i,j]>R 
                @constraint(model,x[i,j]==0)
            end
        end
    end

    for i in eachindex(regions) 
        @constraint(model, sum(sum(x[k,j] for j in 1:n) for k in regions[i] ) >= 1)
    end

    for c in eachindex(contraintes_sous_tours)
        card_c = 0
        for i in 1:n
            if contraintes_sous_tours[c][i] == 1
                card_c = card_c + 1
            end
        end
        @constraint(model, sum(sum(x[i,j]*contraintes_sous_tours[c][i]*contraintes_sous_tours[c][j] for i in 1:n) for j in 1:n) <= card_c - 1)
    end

    #objectif
    @objective(model, Min, sum(sum(D[i,j]*x[i,j] for i in 1:n) for j in 1:n))
    JuMP.optimize!(model) 
    x_val = JuMP.value.(x)
    push!(X,x_val)
    objective = JuMP.objective_value(model)

    contraintes_violees, nouvelle_contrainte = sous_probleme(x_val)
    push!(contraintes_sous_tours, nouvelle_contrainte)
end

print(objective)